### 감성분석
- BERT 모델링 및 모델 성능 테스트

- 사용 라이브러리 로드

In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from konlpy.tag import Okt
from tqdm import tqdm

DEVICE = torch.device("mps")

In [5]:
train_data = pd.read_csv("./data/train_reviews.csv")
test_data = pd.read_csv("./data/test_reviews.csv")
train_data.drop(["Unnamed: 0"], axis=1, inplace=True)
test_data.drop(["Unnamed: 0"], axis=1, inplace=True)

len(train_data), len(test_data)

(8421, 2106)

In [6]:
okt = Okt()

train_data["document"] = train_data["document"].map(lambda x: " ".join(okt.morphs(x, stem=True)))
test_data["document"] = test_data["document"].map(lambda x: " ".join(okt.morphs(x, stem=True)))

In [8]:
train_texts = train_data['document'].astype(str).tolist()
train_labels = train_data['label'].tolist()
test_texts = test_data['document'].astype(str).tolist()
test_labels = test_data['label'].tolist()

In [9]:
model_name = 'monologg/kobert'
tokenizer = BertTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [10]:
class sentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
       item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
       item['labels'] = torch.tensor(self.labels[idx])
       return item

In [12]:
BATCH_SIZE = 64

train_DS = sentimentDataset(train_encodings, train_labels)
test_DS = sentimentDataset(test_encodings, test_labels)

train_DL = torch.utils.data.DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=False)

len(train_DL), len(test_DL)

(132, 33)

In [13]:
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
num_epochs = 10 
learning_rate = 2e-5 #2e-5는 0.00002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()
model.to(DEVICE) # GPU 사용이 가능한 경우

for epoch in range(num_epochs):
    model.train() # 훈련 모드 지정
    total_loss = 0

    for batch in tqdm(train_DL):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_DL)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in test_DL:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)

        correct_predictions += torch.sum(predicted_labels == labels).item()
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
input_text = '뼈다귀 맛집'
input_encoding = tokenizer.encode_plus(
    input_text,
    truncation=True,
    padding=True,
    return_tensors='pt'
)

input_ids = input_encoding['input_ids'].to(DEVICE)
attention_mask = input_encoding['attention_mask'].to(DEVICE)

model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    _, predicted_labels = torch.max(outputs.logits, dim=1)
predicted_labels = predicted_labels.item()

print(predicted_labels)